# Making a Discord Bot

## Initializing Discord Bot Library

In [2]:
import discord

## Initializing Instance of Discord Bot

In [3]:
intents = discord.Intents.default()
intents.message_content = True

client = discord.Client(intents=intents)

## Creating Events

### on_ready

In [ ]:
@client.event
async def on_ready():
    print(f'We have logged in as {client.user}')

### on_message

In [ ]:
@client.event
async def on_message(message):
    
    # Ignore messages from itself
    if message.author == client.user:
        return

    if message.content.startswith('%'):
        message_split = message.content.split()
        command = message_split[0][1:].lower()
        
        # Command for asking help
        if command == "help":
            await message.channel.send(
                'Ask me about schedules in the math department by writing `%ask` followed by your question.'
            )
        
        # Command for closing bot
        elif command == "close":
            await client.close()
        
        # Command for making queries to the bot
        elif command == "ask":
            query = message.content[4:]
            await process_query(message.channel, query)

#### Processing Questions

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from IPython.display import Image
from html2image import Html2Image

In [ ]:
days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
days_id_part = ["sun", "mon", "tue", "wed", "thu", "fri", "sat"]

times = ['7:30 AM - 9:00 AM', '9:00 AM - 10:30 AM', '10:30 AM - 12:00 PM',
         '12:00 PM - 1:30 PM', '1:30 PM - 3:00 PM', '3:00 PM - 4:30 PM', 
         '4:30 PM - 6:00 PM', '6:00 PM - 7:30 PM', '7:30 PM - 9:00 PM']
times_id_part = ["7-30am", "9-00am", "10-30am", "12-00pm", "1-30pm",
                 "3-00pm", "4-30pm", "6-00pm", "7-30pm"]

In [ ]:
async def process_query(channel, query):
    await channel.send("Processing...")
    sched = chatbot.ask(query)
    
    if sched["message"] is not None:
        await channel.send(sched["message"])

    if sched["result"] is not None:
        if len(sched["result"]) < 1: return
            
        fp = open("template.html")
        soup = BeautifulSoup(fp, features="html.parser")
        fp.close()
        
        for i in range(len(sched["result"])):
            row_vals = sched["result"].iloc[i,].values
            ln = row_vals[1]
            room = row_vals[2]
            section = row_vals[3]
            course = row_vals[4]
            day = row_vals[5]
            time = row_vals[6]

            td_prefix = days_id_part[days.index(day)]
            td_suffix = times_id_part[times.index(time)]
            td_id = td_prefix + '-' + td_suffix
            
            if len(soup.find(id = td_id).contents) > 0:
                soup.find(id = td_id).append(BeautifulSoup("<br>", features="html.parser"))
            
            schedule_html = '<p>' + course + '</p><p>' + ln + '</p><p>' + section + '</p><p>' + room + '</p>'
            schedule_element = BeautifulSoup(schedule_html, features="html.parser")
            soup.find(id = td_id).append(schedule_element)

            html_code_txt = open("template_filled.html", "w")
            html_code_txt.write(str(soup))
            html_code_txt.close()

        hti = Html2Image()
        hti.size = (1000, 1000)
        
        # Check if overlap
        time_day_strs = sched["result"]["Time"] + sched["result"]["Day"]
        if len(time_day_strs) != len(time_day_strs.unique()):
            hti.size = (1000, 2250)

        with open('template_filled.html', 'r') as f:
            hti.screenshot(html_str=f.read(), save_as='sched_output.png')

        Image(filename='sched_output.png') 
        with open('sched_output.png', 'rb') as fp:
            if sched["message"] is None:
                await channel.send("Query processed! Here are the results:")
                    
            await channel.send(file=discord.File(fp, 'sched_output.png'))

## Activating Bot

In [ ]:
client.run(SECRET_TOKEN)